In [ ]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, GRU
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.backend import square, mean
from tensorflow.python.keras.initializers import RandomUniform
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau, LearningRateScheduler
import time


SEQ_LEN=720
OUTPUT_LEN=60
EPOCHS=2
STEPS_PER_EPOCH=100
BATCH_SIZE=256
TARGET_VALUE="close"
sequence_length = 60
shift_steps=10
warmup_steps = 50

In [ ]:
# url = "./datasets/DAT_ASCII_EURUSD_M1_2018.csv"
url = "./datasets/eurusd-m15-2018/EURUSD.csv"
df = pd.read_csv(url)
df.drop("Volume", 1, inplace=True)
df.drop("Date", 1, inplace=True)
df.drop("Timestamp", 1, inplace=True)

df.head()

In [ ]:
df.plot(subplots=True, layout=(2, 2), figsize=(40, 20), sharex=False)

In [ ]:
data_set = df.iloc[:, 2:3].values #close
print(data_set)

In [ ]:
print(data_set.shape)

In [ ]:
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(data_set)
training_set_scaled.size

In [ ]:
X_train = []
y_train = []
for i in range(SEQ_LEN, training_set_scaled.size-OUTPUT_LEN):
    X_train.append(training_set_scaled[i-SEQ_LEN:i, 0])
    y_train.append(np.array(training_set_scaled[i:i+OUTPUT_LEN, 0]))
X_train, y_train = np.array(X_train), np.array(y_train)

print(X_train[0])
print(y_train[0])

In [ ]:
X_train = np.reshape(np.asarray(X_train).astype(np.float32), (X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape)

In [ ]:
print(y_train[0])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
def loss_mse_warmup(y_true, y_pred):
    """
    Calculate the Mean Squared Error between y_true and y_pred,
    but ignore the beginning "warmup" part of the sequences.
    
    y_true is the desired output.
    y_pred is the model's output.
    """
    
    print(y_true, y_pred)
    
    # The shape of both input tensors are:
    # [batch_size, sequence_length, num_y_signals].

    # Ignore the "warmup" parts of the sequences
    # by taking slices of the tensors.
    
    y_true_slice = y_true[:, warmup_steps:, :]
    y_pred_slice = y_pred[:, warmup_steps:]
    # These sliced tensors both have this shape:
    # [batch_size, sequence_length - warmup_steps, num_y_signals]
    # Calculat the Mean Squared Error and use it as loss.
    mse = mean(square(y_true_slice - y_pred_slice))
    
    return mse

In [ ]:
model = Sequential()

model.add(GRU(units=512,input_shape=(None, 1)))
init = RandomUniform(minval=-0.05, maxval=0.05)

model.add(Dense(sequence_length, activation='linear', kernel_initializer=init))

# opt = Adam(lr=0.001, decay=1e-6)

NAME = f"PRED-{int(time.time())}-OUTPUT_LEN={OUTPUT_LEN}-SEQ_LEN={SEQ_LEN}"  
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME, update_freq=25))
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=3, min_lr=0.0001)


def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    return lr / (epoch*3)  

lr_scheduler_cb=LearningRateScheduler(schedule=scheduler)

optimizer = RMSprop(lr=1e-3)
model.compile(loss="mse", optimizer=optimizer)

print(model.summary())

In [ ]:

model.fit(X_train,y_train,epochs=EPOCHS,batch_size=256,validation_data=(X_test, y_test), callbacks=[tensorboard])

model_file=f"models/{NAME}_model.h5"

model.save(model_file)

score=model.evaluate(X_test, y_test, verbose=1) 

In [ ]:
print(NAME)

In [ ]:
chain_predict = np.array([training_set_scaled[:SEQ_LEN]])
pd.DataFrame(data=chain_predict[0]).plot(figsize=(20, 15), )

In [ ]:
output=[]

for i in range(SEQ_LEN,5000):
    if i%100==0 or i==SEQ_LEN:
        print(i/5000*100)
    pred_val=model.predict(chain_predict)
    output.append(pred_val)
    concatenated=np.concatenate((np.array([x[0] for x in chain_predict[0][:SEQ_LEN-OUTPUT_LEN]]), pred_val[0]))
    chain_predict=np.array([concatenated.reshape(-1,1)])

In [ ]:
print(model.predict(chain_predict))
print()
print(np.array([x[0] for x in output]).flatten())
output=np.array([x[0] for x in output]).flatten()

In [ ]:
npoutput=sc.inverse_transform(np.array(output).reshape(-1,1))
npoutput=np.array(output).reshape(1,-1)[0]


pd.DataFrame(data=npoutput).plot(figsize=(30, 17))

In [ ]:
print(npoutput)

In [ ]:
pd.DataFrame(data=npoutput[:100]).plot(figsize=(30, 17))

In [ ]:
ytst=model.predict(np.array([X_test[0]]))

In [ ]:
y_tst_np=np.array(y_test[0])
xtest_np=np.array([x[0] for x in X_test[0]])
print(xtest_np)
cont=np.concatenate((xtest_np,y_tst_np ))
print(cont.shape)

In [ ]:
pd.DataFrame(data=cont).plot(figsize=(30, 17))

In [ ]:
# X_train, X_test, y_train, y_test

